**Problem set 1.2 IO: Production Function Estimation OP and ACF**

Author: Jordi Torres 

Start Date: 11/12/2025



In [1]:
using StatFiles, DataFrames

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/GMdata.dta" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = DataFrame(load(data_path))

Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv
,Int16?,Int16?,Int8?,Float32?,Float32?,Float32?,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537
6,2,366,88,5.19584,0.518794,3.46187,3.99812,2.82213,2.59
7,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376
8,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182
9,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101


In [2]:
##1.Ols on the full panel

df= dropmissing(main, [:ldsal, :lemp, :ldnpt]) ##no missings


X= hcat(
    ones(length(df.ldsal)),
    df.ldnpt,
    df.lemp
)

beta_ols= (X'X)\(X'df.ldsal)


3-element Vector{Float64}:
 3.0798222249143237
 0.42719629387166963
 0.5433117916971737

In [3]:
using FixedEffectModels


reg_ols1 = reg(
    df,
    @formula(ldsal ~ ldnpt + lemp),
    Vcov.cluster(:index)
)

                             FixedEffectModel                             
Number of obs:                  2971  Converged:                      true
dof (model):                       2  dof (residuals):                1398
R²:                            0.919  R² adjusted:                   0.919
F-statistic:                 7749.51  P-value:                       0.000
             Estimate  Std. Error   t-stat  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
ldnpt        0.427196   0.0190257  22.4537    <1e-94   0.389874   0.464518
lemp         0.543312   0.0241517  22.4958    <1e-95   0.495934   0.590689
(Intercept)  3.07982    0.0610165  50.4753    <1e-99   2.96013    3.19952


In [4]:
#we define a dataset where only the firms appearing in all years stay. Balanced.

using DataFrames


df1 = dropmissing(df, :yr)
T   = length(unique(df1.yr))

good = combine(groupby(df1, :index)) do sdf
    balanced=
    all(.!ismissing.(sdf.ldsal)) &&
    length(unique(sdf.yr)) == T
    (; keep = balanced)
end

filter_k= good[good.keep, :index]
df_balanced = df1[in.(df1.index, Ref(filter_k)), :]


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv
,Int16?,Int16?,Int8,Float32,Float32,Float32,Float32?,Float32?,Float32?
1,8,355,73,7.30756,3.157,5.67673,4.4641,2.4318,4.5587
2,8,355,78,7.04852,2.91777,5.61488,4.31813,2.59354,3.82598
3,8,355,83,6.47044,1.89462,4.90529,4.5393,1.73744,2.56534
4,8,355,88,6.67208,2.02009,5.25774,4.52065,2.51504,2.5724
5,13,283,73,7.37242,3.05965,6.65504,6.0612,4.49305,4.47722
6,13,283,78,7.87958,3.29595,6.77531,6.35875,4.82749,4.83307
7,13,283,83,8.17725,3.53596,7.23209,6.61925,5.33549,5.75958
8,13,283,88,8.44212,3.65716,7.5052,7.21734,6.0851,6.22851
9,16,280,73,4.1332,-1.60944,1.99029,0.951125,-1.21886,-0.731078


In [5]:
reg_ols2 = reg(
    df_balanced,
    @formula(ldsal ~ ldnpt + lemp),
    Vcov.cluster(:index)
)

                             FixedEffectModel                             
Number of obs:                   856   Converged:                      true
dof (model):                       2   dof (residuals):                 212
R²:                            0.913   R² adjusted:                   0.913
F-statistic:                 1486.57   P-value:                       0.000
             Estimate  Std. Error    t-stat  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
ldnpt        0.533454   0.0355608  15.0012     <1e-34   0.463356   0.603552
lemp         0.428931   0.0483986   8.86247    <1e-15   0.333527   0.524336
(Intercept)  2.72346    0.125047   21.7794     <1e-55   2.47696    2.96995


In [6]:
#first differences

using ShiftedArrays

    df3 = deepcopy(main)

    sort!(df3, [:index, :yr])

    df3 = transform(
        groupby(df, :index),
        :ldsal => (x -> x .- lag(x, 1) ) => :dif_ldsal,
        :ldnpt => (x -> x .- lag(x, 1)) => :dif_ldnpt,
        :lemp => (x -> x .- lag(x, 1)) => :dif_lemp,
    )

    first_dif= dropmissing(df3, [:dif_ldsal, :dif_ldnpt, :dif_lemp])

    reg_ols2 = reg(
    first_dif,
    @formula(dif_ldsal ~ dif_ldnpt + dif_lemp),
    Vcov.cluster(:index)
 )


                              FixedEffectModel                              
Number of obs:                   1571   Converged:                       true
dof (model):                        2   dof (residuals):                  867
R²:                             0.530   R² adjusted:                    0.529
F-statistic:                  256.209   P-value:                        0.000
              Estimate  Std. Error    t-stat  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
dif_ldnpt    0.0542293   0.0291938   1.85756    0.0636  -0.0030695   0.111528
dif_lemp     0.87041     0.0474901  18.3282     <1e-62   0.777201    0.963619
(Intercept)  0.170473    0.0126367  13.4903     <1e-37   0.145671    0.195275


In [7]:
#within group

reg_fe = reg(
    df,
    @formula(ldsal ~ ldnpt + lemp + fe(index)), 
    Vcov.cluster(:index)
)


                          FixedEffectModel                          
Number of obs:               2440   Converged:                   true
dof (model):                    2   dof (residuals):              869
R²:                         0.976   R² adjusted:                0.963
F-statistic:              207.171   P-value:                    0.000
R² within:                  0.536   Iterations:                     1
       Estimate  Std. Error    t-stat  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────
ldnpt  0.31122    0.0368271   8.45085    <1e-15   0.23894    0.383501
lemp   0.752598   0.0606898  12.4007     <1e-31   0.633483   0.871714


In [8]:
using DataFrames, GLM, StatsBase

df.survived .= in.(df1.index, Ref(filter_k)) ##whether the firm survives the four years. 


probit = glm(@formula(survived ~  ldrnd + ldinv), df, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

survived ~ 1 + ldrnd + ldinv

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       z  Pr(>|z|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  -1.34038     0.0492047  -27.24    <1e-99  -1.43682    -1.24394
ldrnd         0.258931    0.0249496   10.38    <1e-24   0.21003     0.307831
ldinv         0.0791476   0.0228622    3.46    0.0005   0.0343386   0.123957
────────────────────────────────────────────────────────────────────────────

**Olly Pakes**



In [9]:
##Olly/Pakes 

#step 1

Kit= df.ldnpt
Iit= df.ldinv
Lit= df.lemp
Yit= df.ldsal


#Flexible function that I can use with choosen number of polynomials K.
function poly2(k, i, l)
    n = length(k)
    X = hcat(
        ones(n),
        l,
        k, i, 
        k.^2, i.^2,
        k .* i, 
        k.^3, i.^3, 
        k.^2 .* i, 
        i.^2 .* k, 
    )
    return X
end

X= poly2(Kit, Iit, Lit)

Beta1= (X'X)\(X'Yit)

beta_l_hat= Beta1[2] #first step output 1

#I need standard errors on this...clustered at the firm level. To code! 

y_hat= X*Beta1

df.ϕ_t_hat= y_hat .- beta_l_hat .* Lit # first step output 2



2971-element Vector{Float64}:
 6.209781430835834
 6.250502801427069
 5.81039690700373
 4.095766380548965
 4.575993155774615
 4.575506937899413
 3.7173814404073697
 4.201416215664165
 4.415133564487948
 4.519976453972198
 4.370914417939908
 5.266391447457371
 4.855595378934307
 ⋮
 6.888389074478899
 6.626477934505273
 4.652605549853603
 3.743532192551039
 4.0060775860474145
 4.599241630536058
 4.805087902977886
 5.189322623894176
 5.524873258596808
 5.636494889143745
 5.6037225846508925
 3.939060228340184

In [10]:
using DataFrames, ShiftedArrays

sort!(df, [:index, :yr])

df2 = transform(
    groupby(df, :index),
    :yr => (x -> lead(x, 1)) => :lead_yr
)

df2.survived= .!ismissing.(df2.lead_yr)

#drop last year of the panel 

data_step2= filter(row->row.yr<88, df2)


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,lead_yr
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32?,Float32?,Bool,Float64,Int8?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,true,6.20978,78
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,true,6.2505,83
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,false,5.8104,missing
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,true,4.09577,83
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,true,4.57599,88
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,true,3.71738,83
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,false,4.20142,missing
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,true,4.41513,78
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,true,4.51998,83


In [11]:
function poly3(k, i)
    n = length(k)
    X = hcat(
        ones(n),
        k, i, 
        k.^2, i.^2,
        k .* i, 
        k.^3, i.^3, 
        k.^2 .* i, 
        i.^2 .* k, 
    )
    return X
end

vars = [:survived, :ldrnd, :ldinv]

df_clean = dropmissing(data_step2, vars)

Iit= df_clean.ldinv
Kit= df_clean.ldnpt
y= df_clean.survived



X= poly3(Iit,Kit)


probit = glm(X, y, Binomial(), ProbitLink())

df_clean.P̂ = predict(probit)

df_clean_small = select(df_clean, :yr, :index, :P̂)


Row,yr,index,P̂
,Int8?,Int16?,Float64
1,73,1,0.813965
2,78,1,0.794649
3,83,1,0.510251
4,78,2,0.58963
5,83,2,0.651903
6,78,3,0.549118
7,83,3,0.587008
8,73,4,0.68706
9,78,4,0.667888


In [12]:
#step 3. 

df_clean


df_total = innerjoin(
    df,
    select(df_clean, :yr, :index, :P̂),
    on = [:index, :yr]
)

#we generate t+1 leads. 
df_total_final= transform(groupby(df_total, :index),
    :ldsal => (x->lead(x)) => :ldsal_lead,
    :lemp => (x->lead(x)) => :lemp_lead,
    :ldnpt => (x->lead(x)) => :ldnpt_lead
)





Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,P̂,ldsal_lead,lemp_lead,ldnpt_lead
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32?,Float32?,Bool,Float64,Float64,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,false,6.20978,0.813965,7.09394,2.67104,7.18584
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,false,6.2505,0.794649,6.4082,2.00647,7.4539
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,false,5.8104,0.510251,missing,missing,missing
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,false,4.09577,0.58963,4.37366,0.640801,3.4784
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,false,4.57599,0.651903,missing,missing,missing
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,false,3.71738,0.549118,3.85115,-0.478036,2.69638
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,false,4.20142,0.587008,missing,missing,missing
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,false,4.41513,0.68706,4.45412,0.330023,3.32176
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,false,4.51998,0.667888,4.19622,0.276115,3.16302


In [13]:

#filter for no missing in these key variables

df_total_final=dropmissing(df_total_final, [:ldsal_lead, :ldnpt_lead, :lemp_lead, :P̂, :ϕ_t_hat , :ldnpt])

#define main variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead
k_t=df_total_final.ldnpt

p_t=df_total_final.P̂
phi_t= df_total_final.ϕ_t_hat

#dependent variable
y= @. y_tlead- beta_l_hat*l_tlead


#polynomial on p_t, phi_t

function poly_final(ωt, p_t)

    n = length(p_t)

    X = hcat(
        ones(n),
        p_t, ωt, 
        p_t.^2, ωt.^2,
        p_t .* ωt, 
        p_t.^3, ωt.^3, 
        p_t.^2 .* ωt, 
        ωt.^2 .* p_t, 
    )
    return X
    

end 


beta_l_hat

0.5246170338398485

In [14]:
function nonlinear_solv(β_k, p_t, y)

            ωt= phi_t .- β_k .* k_t
            y_reg= y .- β_k .* k_tlead
            X= poly_final(ωt, p_t)

            beta_try= (X'X)\(X'y_reg)

            ϵ= y_reg - X*beta_try

            sse = sum(ϵ.^2)
            return sse

end 



nonlinear_solv (generic function with 1 method)

In [16]:
   using Optim
    θ0= [0.4]

   ob1(βk) = nonlinear_solv(βk, p_t, y)

   res1 = optimize(ob1, 0.01, 1.0, Brent())

   β_k = Optim.minimizer(res1)



0.4387653703903427

In [23]:
##adding rd_investment Rt+1​=(1−δR​)Rt​+ItR


##Olly/Pakes 

#step 1

Kit= df.ldnpt
Iit= df.ldinv
Lit= df.lemp
Yit= df.ldsal
RDt= df.ldrnd


#Flexible function that I can use with choosen number of polynomials K.
function poly3(k, i, l, rd)
    n = length(k)
    X = hcat(
        ones(n),
        l,
        k, i, rd,
        k.^2, i.^2, rd.^2, 
        k .* i, k .* rd, i .* rd, 
        k.^3, i.^3, rd.^3,
        k.^2 .* i, k.^2 .* rd,
        i.^2 .* k, i.^2 .* rd,
        rd.^2 .*k, rd.^2 .*i
    )
    return X
end

X= poly3(Kit, Iit, Lit, RDt)

Beta1= (X'X)\(X'Yit)

beta_l_hat_2= Beta1[2] #first step output 1

#I need standard errors on this...clustered at the firm level. To code! 

y_hat= X*Beta1

df.ϕ_t_hat_2= y_hat .- beta_l_hat_2 .* Lit # first step output 2


2971-element Vector{Float64}:
 6.243001884338234
 6.263638107978824
 5.767861892161921
 4.128996421910058
 4.5741923067634715
 4.65586136600125
 3.690843855802307
 4.17442915037667
 4.338544671415202
 4.469421853526029
 4.354152357591335
 5.251546824000859
 4.811319857987152
 ⋮
 6.822896186298927
 6.645492274480067
 4.620860687673068
 3.7196452707041865
 4.005763742139586
 4.618043569567285
 4.807641852548841
 5.162829674915016
 5.580860372645564
 5.71173585054294
 5.715004626078307
 4.2232914497219305

In [26]:
df_clean


df_total = innerjoin(
    df,
    select(df_clean_small, :yr, :index, :P̂),
    on = [:index, :yr]
)

#we generate t+1 leads. 
df_total_final= transform(groupby(df_total, :index),
    :ldsal => (x->lead(x)) => :ldsal_lead,
    :lemp => (x->lead(x)) => :lemp_lead,
    :ldnpt => (x->lead(x)) => :ldnpt_lead,
    :ldrnd => (x->lead(x)) => :ldrnd_lead
)


Row,index,sic3,yr,ldsal,lemp,ldnpt,ldrst,ldrnd,ldinv,survived,ϕ_t_hat,ϕ_t_hat_2,P̂,ldsal_lead,lemp_lead,ldnpt_lead,ldrnd_lead
,Int16?,Int16?,Int8?,Float32,Float32,Float32,Float32?,Float32,Float32,Bool,Float64,Float64,Float64,Float32?,Float32?,Float32?,Float32?
1,1,374,73,7.01896,2.51098,7.04537,3.7509,2.03022,4.98944,false,6.20978,6.243,0.813965,7.09394,2.67104,7.18584,2.04228
2,1,374,78,7.09394,2.67104,7.18584,3.79655,2.04228,4.90343,false,6.2505,6.26364,0.794649,6.4082,2.00647,7.4539,2.0334
3,1,374,83,6.4082,2.00647,7.4539,3.92492,2.0334,3.82753,false,5.8104,5.76786,0.510251,missing,missing,missing,missing
4,2,366,78,3.52628,-0.0523465,2.4056,2.61583,1.12504,0.633175,false,4.09577,4.129,0.58963,4.37366,0.640801,3.4784,1.77073
5,2,366,83,4.37366,0.640801,3.4784,3.22696,1.77073,1.91537,false,4.57599,4.57419,0.651903,missing,missing,missing,missing
6,3,358,78,3.80885,-0.543005,1.72808,0.746074,-1.05687,-0.748376,false,3.71738,3.69084,0.549118,3.85115,-0.478036,2.69638,0.265216
7,3,358,83,3.85115,-0.478036,2.69638,1.32711,0.265216,0.725182,false,4.20142,4.17443,0.587008,missing,missing,missing,missing
8,4,381,73,4.25076,0.210261,3.03931,2.75336,0.812571,2.16101,false,4.41513,4.33854,0.68706,4.45412,0.330023,3.32176,0.366463
9,4,381,78,4.45412,0.330023,3.32176,2.42518,0.366463,2.04567,false,4.51998,4.46942,0.667888,4.19622,0.276115,3.16302,-0.111755


In [28]:

#filter for no missing in these key variables

df_total_final=dropmissing(df_total_final, [:ldsal_lead, :ldnpt_lead, :lemp_lead, :P̂, :ϕ_t_hat_2 , :ldnpt, :ldrnd_lead, :ldrnd])

#define main variables 
y_tlead= df_total_final.ldsal_lead
k_tlead=df_total_final.ldnpt_lead
l_tlead= df_total_final.lemp_lead
rd_tlead= df_total_final.ldrnd_lead

k_t=df_total_final.ldnpt
rd_t=df_total_final.ldrnd

p_t=df_total_final.P̂
phi_t= df_total_final.ϕ_t_hat_2

#dependent variable
y= @. y_tlead- beta_l_hat*l_tlead


#polynomial on p_t, phi_t

function poly_final(ωt, p_t)

    n = length(p_t)

    X = hcat(
        ones(n),
        p_t, ωt, 
        p_t.^2, ωt.^2,
        p_t .* ωt, 
        p_t.^3, ωt.^3, 
        p_t.^2 .* ωt, 
        ωt.^2 .* p_t, 
    )
    return X
    

end 


beta_l_hat

0.5246170338398485

In [29]:
function nonlinear_solv(β, p_t, y)
            β_k, β_rd= β

            ωt= phi_t .- β_k .* k_t - β_rd .* rd_t
            y_reg= y .- β_k .* k_tlead - β_rd.*rd_tlead
            X= poly_final(ωt, p_t)

            beta_try= (X'X)\(X'y_reg)

            ϵ= y_reg - X*beta_try

            sse = sum(ϵ.^2)
            return sse

end 


nonlinear_solv (generic function with 1 method)

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

...

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

TypeError: TypeError: in typeassert, expected REPL.REPLCompletions.MethodCompletion, got a value of type REPL.REPLCompletions.TextCompletion

In [31]:
   using Optim
    θ0= [0.7, 0.9]

   ob1(βk) = nonlinear_solv(βk, p_t, y)

   res1 = optimize(ob1, θ0, NelderMead())

   β= Optim.minimizer(res1)


2-element Vector{Float64}:
 0.38037014580193285
 0.023227021741428114

**Note: this can get better coded... quite rough right now...**